In [1]:
import numpy as np
import pandas as pd
from keras.applications.resnet import ResNet50,preprocess_input
from keras.utils import load_img, img_to_array 
from keras.models import Model


In [2]:
import csv


captions=[]
with open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\results.csv", 'r',encoding="utf8") as csvfile:
  csv_reader = csv.reader(csvfile)
  for row in csv_reader: 
      str1=""
      str1= str1.join(row)
      captions.append(str1)
  csvfile.close()

In [3]:
captions.pop(0)

'image_name| comment_number| comment'

In [4]:
captions

['1000092795.jpg| 0| Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 '1000092795.jpg| 1| Two young  White males are outside near many bushes .',
 '1000092795.jpg| 2| Two men in green shirts are standing in a yard .',
 '1000092795.jpg| 3| A man in a blue shirt standing in a garden .',
 '1000092795.jpg| 4| Two friends enjoy time spent together .',
 '10002456.jpg| 0| Several men in hard hats are operating a giant pulley system .',
 '10002456.jpg| 1| Workers look down from up above on a piece of equipment .',
 '10002456.jpg| 2| Two men working on a machine wearing hard hats .',
 '10002456.jpg| 3| Four men on top of a tall structure .',
 '10002456.jpg| 4| Three men on a large rig .',
 '1000268201.jpg| 0| A child in a pink dress is climbing up a set of stairs in an entry way .',
 '1000268201.jpg| 1| A little girl in a pink dress going into a wooden cabin .',
 '1000268201.jpg| 2| A little girl climbing the stairs to her playhouse .',
 '1000268201.jpg| 3|

In [5]:
dict={}

for x in captions:

    first=x.split("|")[0].split('.')[0]
    try:
        third=x.split("|")[2]
    except:
        third=" "
    if dict.get(first) is None:
        dict[first]=[]
    dict[first].append(third)
   
   
    



In [6]:
dict

{'1000092795': [' Two young guys with shaggy hair look at their hands while hanging out in the yard .',
  ' Two young  White males are outside near many bushes .',
  ' Two men in green shirts are standing in a yard .',
  ' A man in a blue shirt standing in a garden .',
  ' Two friends enjoy time spent together .'],
 '10002456': [' Several men in hard hats are operating a giant pulley system .',
  ' Workers look down from up above on a piece of equipment .',
  ' Two men working on a machine wearing hard hats .',
  ' Four men on top of a tall structure .',
  ' Three men on a large rig .'],
 '1000268201': [' A child in a pink dress is climbing up a set of stairs in an entry way .',
  ' A little girl in a pink dress going into a wooden cabin .',
  ' A little girl climbing the stairs to her playhouse .',
  ' A little girl climbing into a wooden playhouse ',
  ' A girl going into a wooden building .'],
 '1000344755': [' Someone in a blue shirt and hat is standing on stair and leaning against

In [7]:
import re
def clean_text(sentence):
    sentence=sentence.lower()
    sentence=re.sub("[^a-z]+"," ",sentence)
    sentence=sentence.split()
    sentence=[s for s in sentence if len(s)>1]
    sentence=" ".join(sentence)

    return sentence
    

In [8]:
text=clean_text('A cat is playing with basketball # 45')

In [9]:
text

'cat is playing with basketball'

In [10]:
for key,captions in dict.items():
    for i in range(len(captions)):
        captions[i]=clean_text(captions[i])
    

In [11]:
vocab=set()
for key in dict.keys():
    [vocab.update(sentence.split()) for sentence in dict[key]]


In [12]:
len(vocab)

18059

In [13]:
total_words=[]

for key in dict.keys():
    [total_words.append(i) for des in dict[key] for i in des.split()]
len(total_words)

1679070

In [14]:
import collections

counter=collections.Counter(total_words)

freq_dict=counter

In [15]:
len(freq_dict.keys())

18059

In [16]:
sorted_freq_cnt=sorted(freq_dict.items(),reverse=True,key=lambda x:x[1])
threshold=10
sorted_freq_cnt=[x for x in sorted_freq_cnt if x[1]>threshold]
total_words=[x[0] for x in sorted_freq_cnt]

In [17]:
len(total_words)

5118

In [18]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [19]:
data_dir="C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\flickr30k_images"

In [ ]:
import splitfolders
splitfolders.ratio("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\flickr30k_images", output="train,val,test", seed=1337, ratio=(0.7, 0.2,0.1))


Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:09,  9.04s/ files]
Copying files: 7 files [00:09,  1.04 files/s]
Copying files: 16 files [00:09,  2.96 files/s]
Copying files: 25 files [00:09,  5.54 files/s]
Copying files: 34 files [00:09,  8.98 files/s]
Copying files: 43 files [00:09, 13.37 files/s]
Copying files: 52 files [00:09, 18.79 files/s]
Copying files: 60 files [00:09, 23.58 files/s]
Copying files: 69 files [00:09, 30.76 files/s]
Copying files: 79 files [00:10, 39.76 files/s]
Copying files: 87 files [00:10, 44.36 files/s]
Copying files: 95 files [00:10, 49.90 files/s]
Copying files: 105 files [00:10, 57.76 files/s]
Copying files: 115 files [00:10, 65.51 files/s]
Copying files: 124 files [00:10, 67.84 files/s]
Copying files: 132 files [00:10, 64.54 files/s]
Copying files: 140 files [00:10, 66.98 files/s]
Copying files: 148 files [00:11, 65.12 files/s]
Copying files: 155 files [00:11, 66.32 files/s]
Copying files: 162 files [00:11, 66.76 files/s]
Copying fil

In [19]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [21]:
train=load_images_from_folder("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train\\flickr30k_images")

In [22]:
len(train)

22248

In [23]:
train[:5]

[array([[[125, 133, 123],
         [156, 165, 155],
         [146, 156, 144],
         ...,
         [213, 215, 215],
         [211, 213, 213],
         [205, 207, 207]],
 
        [[154, 161, 154],
         [190, 198, 188],
         [175, 184, 174],
         ...,
         [212, 214, 214],
         [211, 213, 213],
         [208, 210, 210]],
 
        [[144, 151, 144],
         [168, 175, 168],
         [166, 173, 166],
         ...,
         [207, 209, 209],
         [207, 209, 209],
         [209, 211, 211]],
 
        ...,
 
        [[206, 209, 207],
         [203, 205, 205],
         [218, 220, 221],
         ...,
         [211, 213, 214],
         [208, 210, 210],
         [210, 212, 212]],
 
        [[209, 211, 211],
         [202, 204, 204],
         [214, 216, 217],
         ...,
         [212, 214, 215],
         [208, 210, 210],
         [208, 210, 210]],
 
        [[210, 212, 212],
         [200, 202, 202],
         [221, 223, 224],
         ...,
         [214, 216, 217],
  

In [20]:
train_name=[filename.split('.')[0] for filename in os.listdir("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train\\flickr30k_images")]

In [23]:
test=load_images_from_folder("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\test\\flickr30k_images")

In [21]:
test_name=[filename.split('.')[0] for filename in os.listdir("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\test\\flickr30k_images")]

In [73]:
train_name=train_name[:10000]

In [74]:
len(train_name)

10000

In [75]:
train_dict={}
for img in train_name:
    for cap in dict[img]:
        train_dict[img]=[]
        newcap="startseq"+cap+"endseq"
        train_dict[img].append(newcap)

train_dict

{'10002456': ['startseqthree men on large rigendseq'],
 '1000268201': ['startseqgirl going into wooden buildingendseq'],
 '1000366164': ['startseqtwo men are cooking mealendseq'],
 '10010052': ['startseqwoman talking on cellphone and wearing rollerskatesendseq'],
 '1001465944': ['startseqseveral people standing outside buildingendseq'],
 '1001573224': ['startseqballet class of five girls jumping in sequenceendseq'],
 '1001633352': ['startseqfour people are jumping from the top of flight of stairsendseq'],
 '1001773457': ['startseqblack dog and spotted dog are fightingendseq'],
 '1001896054': ['startseqman driving john deere tractor on main road in the countryendseq'],
 '100197432': ['startseqwomen are standing outsideendseq'],
 '100207720': ['startseqstanding woman holds pan over cakeendseq'],
 '1002674143': ['startseqyoung girl with pigtails painting outside in the grassendseq'],
 '1003420127': ['startseqfive people are sitting in circle with instrumentsendseq'],
 '100444898': ['start

In [94]:

with open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train_dict.txt", 'wb') as f:
    pickle.dump(train_dict, f)

In [77]:

model=ResNet50(weights="imagenet",input_shape=(224,224,3))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [78]:
intermediate_model = Model(inputs=model.input,
                           outputs=model.layers[-2].output)

In [79]:
 from keras.utils import load_img, img_to_array
def process(path):
   
    test_image = load_img(path, target_size = (224, 224))
    test_image = img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    test_image=preprocess_input(test_image)
    return test_image
    

In [80]:
img=process("C:\\Users\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train\\flickr30k_images\\65567.jpg")
img.shape

(1, 224, 224, 3)

In [81]:
def encode_img(path):
    eimg=process(path)
    feature_vector=intermediate_model.predict(eimg)
    feature_vector=feature_vector.reshape((-1,))
    return feature_vector

In [30]:
encode_img("C:\\Users\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train\\flickr30k_images\\65567.jpg")

1/1 [==============================] - 5s 5s/step


array([0.7401008 , 0.11744083, 0.02729085, ..., 0.05453649, 1.1167022 ,
       0.18546504], dtype=float32)

In [ ]:
encoding_train={}
for img in train_name[:10000]:
    path="C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\train\\flickr30k_images\\"+img+".jpg"
    encoding_train[img]=encode_img(path)
    
    

1/1 [==============================] - ETA: 0s

In [32]:
encoding_test={}
for img in test_name:
    path="C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\test\\flickr30k_images\\"+img+".jpg"
    encoding_test[img]=encode_img(path)

1/1 [==============================] - 0s 282ms/step


In [105]:
encoding_test


{'1000344755': array([0.03429769, 0.0598781 , 0.        , ..., 0.29731876, 0.        ,
        0.41135123], dtype=float32),
 '1000523639': array([0.        , 1.6437693 , 0.        , ..., 0.22675855, 2.1350715 ,
        1.5995198 ], dtype=float32),
 '1000919630': array([0.3702    , 0.        , 0.07301956, ..., 0.36130932, 0.6858818 ,
        0.18551895], dtype=float32),
 '101262930': array([0.04609485, 0.9302067 , 0.        , ..., 0.30915684, 0.09157311,
        0.1955798 ], dtype=float32),
 '101362650': array([0.54173064, 4.3220344 , 0.18306588, ..., 0.04345481, 0.610734  ,
        1.6018388 ], dtype=float32),
 '10188041': array([1.401852  , 1.7816921 , 0.04904242, ..., 0.01924163, 1.3168346 ,
        0.        ], dtype=float32),
 '1020651753': array([1.0470679 , 0.45183775, 0.44739556, ..., 1.5936285 , 0.        ,
        0.07959579], dtype=float32),
 '1021293940': array([0.        , 0.17286538, 0.47379878, ..., 0.09630285, 1.4736332 ,
        0.654423  ], dtype=float32),
 '1028205764

In [97]:


import pickle 

with open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\encode_train.txt", 'wb') as f:
    pickle.dump(encoding_train, f)

In [37]:
encoding_train

{'10002456': array([0.44910964, 0.14129679, 0.5372808 , ..., 1.7456448 , 1.4152648 ,
        0.        ], dtype=float32),
 '1000268201': array([0.0653657 , 0.16782762, 0.32518452, ..., 0.05111527, 0.32817978,
        1.0043336 ], dtype=float32),
 '1000366164': array([0.06212308, 0.7190827 , 1.8226362 , ..., 0.23470731, 0.04348029,
        0.94410557], dtype=float32),
 '10010052': array([0.12822375, 0.16603696, 0.27673402, ..., 0.16896215, 0.27030873,
        0.15224312], dtype=float32),
 '1001465944': array([0.56625056, 1.637548  , 0.58604205, ..., 0.7260785 , 0.91917676,
        3.0001967 ], dtype=float32),
 '1001573224': array([0.73310727, 0.14019383, 0.00351705, ..., 1.1137248 , 0.0992556 ,
        0.04274536], dtype=float32),
 '1001633352': array([0.8689903 , 3.4701986 , 0.        , ..., 0.6842563 , 0.34780157,
        0.41004032], dtype=float32),
 '1001773457': array([0.78423005, 0.8189705 , 0.06491412, ..., 0.08858134, 0.6853727 ,
        0.18749578], dtype=float32),
 '1001896054

In [82]:
word_to_indx={}
indx_to_word={}

for indx,word in enumerate(total_words):
    word_to_indx[word]=indx
    indx_to_word[indx]=word

In [83]:
len(indx_to_word)

5118

In [84]:
word_to_indx['startseq']=5119
indx_to_word[5119]='startseq'

word_to_indx['endseq']=5120
indx_to_word[5120]='endseq'

In [67]:
with open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\word_to_indx.txt", 'wb') as f:
    pickle.dump(word_to_indx, f)

In [85]:
maxlen=0
for key in train_dict.keys():
    for cap in train_dict[key]:
        maxlen=max(maxlen,len(cap.split()))
maxlen

32

In [86]:
def data_generator(train_dict,encoding_train,word_to_indx,max_len,batch_size):
    X1,X2,y=[],[],[]
    n=0
    while True:
        for key,desc_list in train_dict.items():
            n+=1
            photo=encoding_train[key+'.jpg']
            for desc in desc_list:
                seq=[word_to_indx[word] for word in desc.split() if word in word_to_indx]
                for i in range(len(seq)):
                    xi=seq[0:i]
                    yi=seq[i]
                    xi=pad_sequences([xi],maxlen=maxlen,value=0,padding='post')[0]
                    yi=to_categorical([yi],num_classes=vocab_size)[0]
                    X1.append(photo)
                    X2.append(xi)
                    y.append(yi)
                if(n==batch_size):
                    yeild [[np.array(X1),np.array(X2)],np.array(y)]
                    X1,X2,y=[],[],[]
                    n=0
        
            

In [88]:
f=open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\glove embeddings\\glove.6B.50d.txt",encoding='utf-8')

In [89]:
embedding_indx={}

for line in f:
    values=line.split()
    word=values[0]
    word_embedding=np.array(values[1:],dtype='float')
    embedding_indx[word]=word_embedding


In [90]:
embedding_indx['apple']

array([ 0.52042 , -0.8314  ,  0.49961 ,  1.2893  ,  0.1151  ,  0.057521,
       -1.3753  , -0.97313 ,  0.18346 ,  0.47672 , -0.15112 ,  0.35532 ,
        0.25912 , -0.77857 ,  0.52181 ,  0.47695 , -1.4251  ,  0.858   ,
        0.59821 , -1.0903  ,  0.33574 , -0.60891 ,  0.41742 ,  0.21569 ,
       -0.07417 , -0.5822  , -0.4502  ,  0.17253 ,  0.16448 , -0.38413 ,
        2.3283  , -0.66682 , -0.58181 ,  0.74389 ,  0.095015, -0.47865 ,
       -0.84591 ,  0.38704 ,  0.23693 , -1.5523  ,  0.64802 , -0.16521 ,
       -1.4719  , -0.16224 ,  0.79857 ,  0.97391 ,  0.40027 , -0.21912 ,
       -0.30938 ,  0.26581 ])

In [56]:
f.close()

In [91]:
def get_embedding_matrix():
    emb_dim=50
    matrix=np.zeros((len(vocab)+1,emb_dim))
    for word,indx in word_to_indx.items():
        embedding_vector=embedding_indx.get(word)
        if embedding_vector is not None:
            matrix[indx]=embedding_vector
    return matrix
            
        
        

In [92]:
embedding_matrix=get_embedding_matrix()

In [93]:
embedding_matrix.shape

(18060, 50)

In [64]:

with open("C:\\Users\\91954\\OneDrive\\Desktop\\datasets\\flickr30k_images\\embedding_matrix.txt", 'wb') as f:
    pickle.dump(embedding_matrix, f)

In [72]:
encoding_train.items()

dict_items([('10002456', array([0.44910964, 0.14129679, 0.5372808 , ..., 1.7456448 , 1.4152648 ,
       0.        ], dtype=float32)), ('1000268201', array([0.0653657 , 0.16782762, 0.32518452, ..., 0.05111527, 0.32817978,
       1.0043336 ], dtype=float32)), ('1000366164', array([0.06212308, 0.7190827 , 1.8226362 , ..., 0.23470731, 0.04348029,
       0.94410557], dtype=float32)), ('10010052', array([0.12822375, 0.16603696, 0.27673402, ..., 0.16896215, 0.27030873,
       0.15224312], dtype=float32)), ('1001465944', array([0.56625056, 1.637548  , 0.58604205, ..., 0.7260785 , 0.91917676,
       3.0001967 ], dtype=float32)), ('1001573224', array([0.73310727, 0.14019383, 0.00351705, ..., 1.1137248 , 0.0992556 ,
       0.04274536], dtype=float32)), ('1001633352', array([0.8689903 , 3.4701986 , 0.        , ..., 0.6842563 , 0.34780157,
       0.41004032], dtype=float32)), ('1001773457', array([0.78423005, 0.8189705 , 0.06491412, ..., 0.08858134, 0.6853727 ,
       0.18749578], dtype=float32)), 

In [99]:
from keras.models import load_model

In [191]:
model=load_model("C:\\Users\91954\\OneDrive\\Desktop\Models\\Image Captioning\\model_19.h5",compile=False)

In [192]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 50)       903000      ['input_2[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_1[0][0]']                
                                                                                              

In [155]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


In [194]:
def predict_captions(photo):
    in_text='startseq'
    arr=[]
    for i in range(maxlen):
        sequence=[word_to_indx[w] for w in in_text.split() if w in word_to_indx]
        sequence=pad_sequences([sequence],maxlen=maxlen,padding='post')
        ypred=model.predict([photo,sequence])
        ypred=ypred.argmax()
        word=indx_to_word[ypred]
        if word not in arr:
            arr.append(word)
            in_text+=(' '+word)
        if word=='endseq':
            break
    final_caption=in_text.split()[1:-1]
    final_caption=" ".join(final_caption)
    return final_caption
            
    

In [ ]:
for i in range(15):
    indx=np.random.randint(0,1000)
    all_img_names=list(encoding_train.keys())
    img_name=all_img_names[indx]
    photo_2048=encoding_train[img_name].reshape((1,2048))
    caption=predict_captions(photo_2048)
    print(img_name)
    print(caption)
    
    